In [ ]:
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

# combining 1st and 2nd half batchAudioSummary

In [12]:
import os
import time
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

# Parameters
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = "1Esy8YW-t2B3-8uwhaFrDD2-1fgzcXJ31hyi97DVPKXI"
CREDENTIALS_FILE = "D:\\Vaani\\Automate\\client_secret_70065301055-fjcq0nugl0id8k0oab6qjse8e3o24in6.apps.googleusercontent.com.json"
TOKEN_FILE = "token.json"
RAW_SHEET_NAME = "rawAuto"
TARGET_SHEET_NAME = "BatchAudioSummaryAuto"
START_FORMULA_COLUMN = ord('E')
END_FORMULA_COLUMN = ord('Z')
START_UPDATE_COLUMN = ord('C')
RATE_LIMIT_DELAY = 0.5

STATUS_DATA = [
    'Raw Delivered',
    'Delivered greater than acceptance threshold',
    'Raw Redelivery',
    'Redelivered greater than acceptance threshold',
    'Accepted post Initial Check (file level)',
    'Accepted post Initial check (chunk level)',
    'Accepted post automated single audio check (chunk level)',
    'Delivered for manual QC',
    'Accepted post final single Audio Manual QC (chunk level)'
]

# Function to authenticate and create a Google Sheets API service instance
def get_sheets_service():
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        return service
    except HttpError as err:
        print(err)
        return None

# Function to check if a sheet exists
def sheet_exists(service, spreadsheet_id, sheet_name):
    try:
        sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        sheets = sheet_metadata.get('sheets', [])
        for sheet in sheets:
            if sheet.get("properties", {}).get("title") == sheet_name:
                return True
        return False
    except HttpError as err:
        print(err)
        return False

def create_or_update_sheet(service, spreadsheet_id, sheet_name):
    # Create a DataFrame with the status data
    df = pd.DataFrame({'Status': STATUS_DATA})

    # Convert DataFrame to list of lists
    values = [df.columns.tolist()] + df.values.tolist()

    # Check if the sheet exists and create if not
    if not sheet_exists(service, spreadsheet_id, sheet_name):
        add_sheet_request = {
            "requests": [
                {
                    "addSheet": {
                        "properties": {
                            "title": sheet_name
                        }
                    }
                }
            ]
        }
        service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=add_sheet_request).execute()

    # Write the DataFrame to the sheet
    body = {
        "values": values
    }
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_name}!A1",
        valueInputOption="RAW",
        body=body
    ).execute()
    time.sleep(RATE_LIMIT_DELAY)

    # Determine the number of columns in the 'raw' sheet dynamically
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=f"{RAW_SHEET_NAME}!2:2"  # Read the second row to get column headers
    ).execute()
    raw_headers = result.get("values", [])
    if raw_headers:
        num_columns_raw = len(raw_headers[0])
    else:
        num_columns_raw = 0

    # Loop through columns from 'E' to 'Z' for FORMULA_COLUMN and 'C' onwards for COL_TO_UPDATE
    for col_offset in range(min(END_FORMULA_COLUMN - START_FORMULA_COLUMN + 1, num_columns_raw)):
        formula_column = chr(START_FORMULA_COLUMN + col_offset)
        col_to_update = chr(START_UPDATE_COLUMN + col_offset)

        # Read the header of the formula column from the 'raw' sheet
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=f"{RAW_SHEET_NAME}!{formula_column}2"  # 2 because in raw sheet the header is in the 2nd row
        ).execute()
        raw_header = result.get("values", [[]])[0][0] if result.get("values") and len(result.get("values")) > 0 else ""

        # Update the header for the new column in 'BatchAudioSummary'
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=f"{sheet_name}!{col_to_update}1",  # 1 because we are adding the header of raw in the 1st row
            valueInputOption="RAW",
            body={"values": [[raw_header]]}
        ).execute()
        time.sleep(RATE_LIMIT_DELAY)

        # Add formulas to the new column
        for i in range(2, 11):
            if i == 9:
                formula = f'={col_to_update}7-{col_to_update}8'
            elif i == 10:
                formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{formula_column}{i}:{formula_column}1000)-ROW({RAW_SHEET_NAME}!{formula_column}{i}),8)=0)*{RAW_SHEET_NAME}!{formula_column}{i}:{formula_column}1000)/60'
            else:
                formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{formula_column}{i+1}:{formula_column}1000)-ROW({RAW_SHEET_NAME}!{formula_column}{i+1}),8)=0)*{RAW_SHEET_NAME}!{formula_column}{i+1}:{formula_column}1000)/60'

            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f"{sheet_name}!{col_to_update}{i}",
                valueInputOption="USER_ENTERED",
                body={"values": [[formula]]}
            ).execute()
            time.sleep(RATE_LIMIT_DELAY)

    # Add a new column for "# of Hours" with row sums
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_name}!B1",
        valueInputOption="RAW",
        body={"values": [["# of Hours"]]}
    ).execute()
    for i in range(2, 11):
        formula = f'=SUM({sheet_name}!C{i}:{chr(START_UPDATE_COLUMN + col_offset)}{i})'
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=f"{sheet_name}!B{i}",
            valueInputOption="USER_ENTERED",
            body={"values": [[formula]]}
        ).execute()
        time.sleep(RATE_LIMIT_DELAY)

def additional_operations(service):
    BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_C_START_ROW = 14
    BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_D_START_ROW = 14
    BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_E_START_ROW = 14

    # Add headers to the 12th row of BatchAudioSummaryAuto sheet
    headers = [
        ["Accepted post Initial check (chunk level)", 
         "Accepted post automated single audio check (chunk level)", 
         "Accepted post final single Audio Manual QC (chunk level)"]
    ]
    
    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range="BatchAudioSummaryAuto!C12:E12",
        valueInputOption='RAW',
        body={'values': headers}
    ).execute()

    # Get the data from the rawAuto sheet
    raw_auto_data = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID,
        range="rawAuto!A3:B"
    ).execute().get('values', [])

    # Extract unique districts and their states
    district_state_map = {}
    for row in raw_auto_data:
        if len(row) < 2:
            continue
        state, district = row
        if district not in district_state_map:
            district_state_map[district] = state

    # Prepare the data to be written to BatchAudioSummaryAuto sheet
    data_to_write = []
    for district, state in district_state_map.items():
        data_to_write.append([state, district])

    # Write the data to the BatchAudioSummaryAuto sheet
    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range="BatchAudioSummaryAuto!A14:B",
        valueInputOption='RAW',
        body={'values': data_to_write}
    ).execute()

    # Prepare and write formulas to the BatchAudioSummaryAuto sheet for column C
    formulas_c_to_write = []
    row_index_c = 8
    for i in range(len(raw_auto_data)):
        formula = f"=ROUND(rawAuto!D{row_index_c}/60,2)"
        formulas_c_to_write.append([formula])
        row_index_c += 8

    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range="BatchAudioSummaryAuto!C14:C",
        valueInputOption='USER_ENTERED',
        body={'values': formulas_c_to_write}
    ).execute()

    # Prepare and write formulas to the BatchAudioSummaryAuto sheet for column D
    formulas_d_to_write = []
    row_index_d = 9
    for i in range(len(raw_auto_data)):
        formula = f"=ROUND(rawAuto!D{row_index_d}/60,2)"
        formulas_d_to_write.append([formula])
        row_index_d += 8

    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range="BatchAudioSummaryAuto!D14:D",
        valueInputOption='USER_ENTERED',
        body={'values': formulas_d_to_write}
    ).execute()

    # Prepare and write formulas to the BatchAudioSummaryAuto sheet for column E
    formulas_e_to_write = []
    row_index_e = 10
    for i in range(len(raw_auto_data)):
        formula = f"=ROUND(rawAuto!D{row_index_e}/60,2)"
        formulas_e_to_write.append([formula])
        row_index_e += 8

    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range="BatchAudioSummaryAuto!E14:E",
        valueInputOption='USER_ENTERED',
        body={'values': formulas_e_to_write}
    ).execute()

    # Prepare and write IF formulas to column G
    if_formulas_to_write = []
    for i in range(len(raw_auto_data)):
        formula = f"=IF(C{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_C_START_ROW + i} > 100, \"Exceeded\", C{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_C_START_ROW + i})"
        if_formulas_to_write.append([formula])

    if_range = f"BatchAudioSummaryAuto!G{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_C_START_ROW}:G{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_C_START_ROW + len(raw_auto_data) - 1}"

    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=if_range,
        valueInputOption='USER_ENTERED',
        body={'values': if_formulas_to_write}
    ).execute()

    # Prepare and write IF formulas to column H based on column D values
    if_formulas_h_to_write = []
    for i in range(len(raw_auto_data)):
        formula = f"=IF(D{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_D_START_ROW + i} > 100, \"Exceeded\", D{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_D_START_ROW + i})"
        if_formulas_h_to_write.append([formula])

    if_range_h = f"BatchAudioSummaryAuto!H{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_D_START_ROW}:H{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_D_START_ROW + len(raw_auto_data) - 1}"

    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=if_range_h,
        valueInputOption='USER_ENTERED',
        body={'values': if_formulas_h_to_write}
    ).execute()
    
    # Prepare and write IF formulas to column I based on column E values
    if_formulas_i_to_write = []
    for i in range(len(raw_auto_data)):
        formula = f"=IF(E{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_E_START_ROW + i} > 100, \"Exceeded\", E{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_E_START_ROW + i})"
        if_formulas_i_to_write.append([formula])

    if_range_i = f"BatchAudioSummaryAuto!I{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_E_START_ROW}:I{BATCH_AUDIO_SUMMARY_AUTO_FORMULAS_E_START_ROW + len(raw_auto_data) - 1}"
    
    #print(if_formulas_i_to_write)
    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=if_range_i,
        valueInputOption='USER_ENTERED',
        body={'values': if_formulas_i_to_write}
    ).execute()
    
    # Add sum formulas to row 13
    sum_formulas = [
        ["=SUM(C14:C93)", "=SUM(D14:D93)", "=SUM(E14:E93)"]
    ]
    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range="BatchAudioSummaryAuto!C13:E13",
        valueInputOption='USER_ENTERED',
        body={'values': sum_formulas}
    ).execute()

def main():
    service = get_sheets_service()
    if service:
        create_or_update_sheet(service, SPREADSHEET_ID, TARGET_SHEET_NAME)
        additional_operations(service)

if __name__ == "__main__":
    main()


# merge the above and the audio_to_raw 